# Quantization Aware Training Sample Code

In [1]:
import os
import random

import torch
import torch.nn as nn
import torchvision

import time
import copy
import numpy as np
from torchvision import transforms
from tqdm import tqdm
from torchsummary import summary
torch.backends.cudnn.benchmark = True

def set_random_seeds(random_seed=0):

    torch.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

def memory_check():
    print(f"  Allocated: {round(torch.cuda.memory_allocated()/1024**3,2)} GB")
    print(f"  Cached:    {round(torch.cuda.memory_reserved()/1024**3,2)} GB\n")

print(f"torch = {torch.__version__}")
print(f"torchvision = {torchvision.__version__}")


torch = 1.12.1
torchvision = 0.13.1


## Make ImageNet(validation 6G) Data Loader 

In [2]:
import wget
def ImageNet_DataLoader(split_num = [0.08,0.02,0.9]):
    if not os.path.exists("./data/ImageNet/meta.bin"):
        print("Meta data download")
        wget.download(url="https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz", out="./data/ImageNet")
    # if not os.path.exists("./data/ImageNet/ILSVRC2012_devkit_t3.tar.gz"):
    #     print("Toolkit t3 Download")
    #     toolkit_url = "https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t3.tar.gz"
    #     wget.download(url= toolkit_url,out="./data/ImageNet")
    if not os.path.exists("./data/ImageNet/ILSVRC2012_img_val.tar"):
        print("Download val data")
        val_url  = 'https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar'
        wget.download(url=val_url, out="./data/ImageNet")

    # if not os.path.exists("./data/ImageNet/ILSVRC2012_img_train_t3.tar"):
    #     print("Download train t3 data")
    #     train_url = "https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train_t3.tar"
    #     wget.download(url=train_url,out="./data/ImageNet")
    train_transform = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])

    test_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])
    print(os.getcwd())
    dataset = torchvision.datasets.ImageNet(root="./data/ImageNet",split="val", transform = train_transform)
    Train_dataset, Test_dataset,_ = torch.utils.data.random_split(dataset, split_num)
    print(f"Train data set = {len(Train_dataset)}, Test = {len(Test_dataset)}")
    
    train_sampler = torch.utils.data.RandomSampler(Train_dataset)
    test_sampler = torch.utils.data.SequentialSampler(Test_dataset)

    Train_loader = torch.utils.data.DataLoader(dataset=Train_dataset, batch_size= 32, sampler = train_sampler)
    Test_loader = torch.utils.data.DataLoader(dataset=Test_dataset, batch_size =32, sampler = test_sampler)
    return Train_loader, Test_loader

In [3]:
def Cifar10_Dataloader():
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding = 4),
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    test_transform = transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])


    train_dataset = torchvision.datasets.CIFAR10(root="data", train=True, download=True, transform=train_transform) 
    # We will use test set for validation and test in this project.
    # Do not use test set for validation in practice!
    test_dataset = torchvision.datasets.CIFAR10(root="data", train=False, download=True, transform=test_transform)
    print(f"Train data set = {len(train_dataset)}, Test = {len(test_dataset)}")

    train_sampler = torch.utils.data.RandomSampler(train_dataset)
    test_sampler = torch.utils.data.SequentialSampler(test_dataset)

    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=128,
        sampler=train_sampler)

    test_loader = torch.utils.data.DataLoader(
        dataset=test_dataset, batch_size=128,
        sampler=test_sampler)
    return train_loader, test_loader


## MobileNetV2

# Train and Evaluate Fuc

In [4]:
def Evaluating(model, test_loader, device, criterion=None):
    model.to(device)
    model.eval()

    running_loss = 0
    running_corrects = 0

    for inputs, labels in tqdm(iter(test_loader)):
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        _, preds = torch.max(outputs, 1)
        if criterion is not None:
            loss = criterion(outputs, labels).item()
        else:
            loss = 0
        # statistics
        running_loss += loss * labels.size(0)
        running_corrects += (preds == labels).sum().item()

    eval_loss = running_loss / len(test_loader.dataset)
    eval_accuracy = 100 * running_corrects / len(test_loader.dataset)

    return eval_loss, eval_accuracy

In [5]:
def Training(model, train_loader, test_loader, device, optimizer, scheduler, epochs=100,model_name="test"):
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    print("Before Training")
    torch.cuda.memory_reserved()
    memory_check()
    count = 0
    best_loss = np.Inf
    # Training
    model.to(device)
    for epoch in range(epochs):

        running_loss = 0
        running_corrects = 0
        model.train()

        for inputs, labels in tqdm(iter(train_loader)):

            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)

            loss = criterion(outputs, labels)
 
            loss.backward()
            optimizer.step()
            
            _, preds = torch.max(outputs, 1)
            # statistics
            running_loss += loss.item() * labels.size(0)
            running_corrects += (preds == labels).sum().item()

        # Set learning rate scheduler
        if scheduler is not None:
            scheduler.step()
        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = 100 * running_corrects / len(train_loader.dataset) 

        # Evaluation
        val_loss, val_acc = Evaluating(model,test_loader,device=device,criterion=criterion)
        print(f"--------{epoch+1}----------")
        print(f"Train {train_loss:.4f} Loss, {train_accuracy:.2f} Acc")
        print(f"Validation {val_loss:.4f} Loss, {val_acc:.2f} Acc")

        if best_loss > val_loss:
            best_loss = val_loss
            count = 0
            torch.save(model.state_dict(), f"./models/{model_name}.pt")
        else:
            count +=1
            if count > 10:
                break
    model.load_state_dict(torch.load(f"./models/{model_name}.pt")) 
    return model

## Layer fusion Check
conv, bn, relu를 하나의 layer로 만들어 각각의 layer를 읽어오는 연산을 줄이는 과정   
folding과는 다른 경량화 기법   
Fusion 된 layer는 identity로 바뀜

In [6]:
def model_eq_check(model1, model2, device, rtol=1e-03, atol=1e-06, num_tests=100, input_size=(1,3,32,32)):

    model1.to(device)
    model2.to(device)

    for _ in range(num_tests):
        x = torch.rand(size=input_size).to(device)
        y1 = model1(x).detach().cpu().numpy()
        y2 = model2(x).detach().cpu().numpy()
        # 배열이 허용 오차범위 abs(a - b) <= (atol + rtol * absolute(b)) 이내면 True
        if np.allclose(a=y1, b=y2, rtol=rtol, atol=atol, equal_nan=False) == False:
            print("Model equivalence test fail")
            return False
    print("Two models equal")
    return True

In [7]:
def time_test(model, device, input_size = (1,3,256,256),num_tests=100,):
    model.to(device)
    model.eval()

    x = torch.rand(size=input_size).to(device)

    with torch.no_grad():
        for _ in range(10):
            _ = model(x)
    torch.cuda.synchronize()

    with torch.no_grad():
        start_time = time.time()

        for _ in range(num_tests):
            _ = model(x)
            torch.cuda.synchronize()
        total_time = time.time() - start_time

    aver_time = total_time / num_tests
    return total_time, aver_time

In [8]:
class ConvBnReLUModel(nn.Module):
    def __init__(self):
        super(ConvBnReLUModel,self).__init__()
        self.conv = nn.Conv2d(3,5,3,bias=True).to(dtype=torch.float)
        self.bn = nn.BatchNorm2d(5).to(dtype=torch.float)
        self.relu = torch.nn.ReLU(inplace=True)
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()
    
    def forward(self,x):
        x = self.quant(x)
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.dequant(x)
        return x
    
model = ConvBnReLUModel().to(device=torch.device("cpu:0"))
model.eval()
print(model)
# for p in model.named_parameters():
#     print(p)
#     print()
# "fbgemm" for server , "qnnpack" for mobile 
# model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# # torch.quantization.fuse_module or myModel.fuse_model()
fuse_model = torch.ao.quantization.fuse_modules(model,[['conv','bn','relu']], inplace=False)
# fuse_model = model.fuse_model()
print(fuse_model)

print(f"-- Equal Test --")
model_eq_check(model, fuse_model, device=torch.device("cpu:0"))


print(f"-- Infer Time Test --")
ori_cpu_time,_ = time_test(model,torch.device("cpu"))
fus_cpu_time,_ = time_test(fuse_model,torch.device("cpu"))

print(f"origin model infer time {ori_cpu_time:.3f}s")
print(f"fusion model infer time {fus_cpu_time:.3f}s")
del model
del fuse_model



ConvBnReLUModel(
  (conv): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
  (bn): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (quant): QuantStub()
  (dequant): DeQuantStub()
)
ConvBnReLUModel(
  (conv): ConvReLU2d(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (bn): Identity()
  (relu): Identity()
  (quant): QuantStub()
  (dequant): DeQuantStub()
)
-- Equal Test --
Two models equal
-- Infer Time Test --
origin model infer time 0.058s
fusion model infer time 0.044s


# MAIN

In [9]:
# gpu,cpu device 선언
if torch.cuda.is_available():
    gpu_device = torch.device("cuda")
cpu_device = torch.device("cpu")

set_random_seeds(42)

# model 가져오기
from models import mobilenet_v2, MobileNet_V2_Weights,quat_mobilenet_v2
model = quat_mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1,activation_layer=nn.ReLU)
model.classifier.append(nn.Dropout(0.2))
model.classifier.append(nn.Linear(1000, 10))

from torchsummary import summary
summary(model,(3,32,32), device='cpu') 

# Move the model to CPU since static quantization does not support CUDA currently.
# ImageNet Data 
Train_loader, Test_loader = Cifar10_Dataloader()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         QuantStub-1            [-1, 3, 32, 32]               0
            Conv2d-2           [-1, 32, 16, 16]             864
       BatchNorm2d-3           [-1, 32, 16, 16]              64
              ReLU-4           [-1, 32, 16, 16]               0
            Conv2d-5           [-1, 32, 16, 16]             288
       BatchNorm2d-6           [-1, 32, 16, 16]              64
              ReLU-7           [-1, 32, 16, 16]               0
            Conv2d-8           [-1, 16, 16, 16]             512
       BatchNorm2d-9           [-1, 16, 16, 16]              32
QuantizableInvertedResidual-10           [-1, 16, 16, 16]               0
           Conv2d-11           [-1, 96, 16, 16]           1,536
      BatchNorm2d-12           [-1, 96, 16, 16]             192
             ReLU-13           [-1, 96, 16, 16]               0
           Conv2d-14         

In [10]:
for i,data in enumerate(Train_loader):

    img = data[0].to(gpu_device)
    label = data[1].to(gpu_device)
    print(f"{i} data size = {img.size()}, label size = {label.size()}")
    memory_check()
    if i > 20:
        break

0 data size = torch.Size([128, 3, 32, 32]), label size = torch.Size([128])
  Allocated: 0.0 GB
  Cached:    0.02 GB

1 data size = torch.Size([128, 3, 32, 32]), label size = torch.Size([128])
  Allocated: 0.0 GB
  Cached:    0.02 GB

2 data size = torch.Size([128, 3, 32, 32]), label size = torch.Size([128])
  Allocated: 0.0 GB
  Cached:    0.02 GB

3 data size = torch.Size([128, 3, 32, 32]), label size = torch.Size([128])
  Allocated: 0.0 GB
  Cached:    0.02 GB

4 data size = torch.Size([128, 3, 32, 32]), label size = torch.Size([128])
  Allocated: 0.0 GB
  Cached:    0.02 GB

5 data size = torch.Size([128, 3, 32, 32]), label size = torch.Size([128])
  Allocated: 0.0 GB
  Cached:    0.02 GB

6 data size = torch.Size([128, 3, 32, 32]), label size = torch.Size([128])
  Allocated: 0.0 GB
  Cached:    0.02 GB

7 data size = torch.Size([128, 3, 32, 32]), label size = torch.Size([128])
  Allocated: 0.0 GB
  Cached:    0.02 GB

8 data size = torch.Size([128, 3, 32, 32]), label size = torch.S

In [11]:
need_train=False
if need_train:
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,60,90], gamma=0.5)

    model = Training(model,train_loader=Train_loader,test_loader=Test_loader,device=gpu_device,optimizer=optimizer,scheduler=scheduler,epochs=20,
    model_name = "q_mobilenetv2_cifar10")
else:
    model.load_state_dict(torch.load("./models/q_mobilenetv2_cifar10.pt"))
    _,pre_acc = Evaluating(model,Test_loader,cpu_device)
    print(f"pretrained model acc : {pre_acc:.2f} %")
    # QAT가 적용된 floating point 모델을 quantized int model로 변환
model.to(cpu_device)

100%|██████████| 79/79 [00:05<00:00, 15.53it/s]

pretrained model acc : 90.06 %


QuantizableMobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): QuantizableInvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (2): QuantizableInvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): C

In [12]:

# 모델을 CPU상태로 두고 eval로 layer fusion
model.eval()
print(model)
# Layer fusion
# fused_model = torch.quantization.fuse_modules(model,[["conv1","bn1","relu"]])

# for module_name, module in fused_model.named_children():
#     if "layer" in module_name:
#         # basic_block 의 conv1, bn1, relu, conv2, bn2 를 fusion
#         for basic_block_name, basic_block in module.named_children():
#             torch.ao.quantization.fuse_modules(basic_block,[["conv1","bn1","relu"],["conv2","bn2"]],inplace=True)
#             # basic_block안의 downsampling block의 Conv2d Batchnorm2D fusion
#             for sub_block_name, sub_block in basic_block.named_children():
#                 if sub_block_name == "downsample":
#                     torch.ao.quantization.fuse_modules(sub_block,[["0","1"]], inplace=True)
# print(fused_model)
fused_model = copy.deepcopy(model)
fused_model.fuse_model()
# Equal Test
print(f"Equal Test between origin and fused")
print(model_eq_check(model,fused_model,device=cpu_device))

QuantizableMobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): QuantizableInvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (2): QuantizableInvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): C

In [25]:
def calibrate_model(model, loader, device=torch.device("cpu")):
    print("calibrating ...")
    model.to(device)
    model.eval()

    for inputs, labels in tqdm(loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        _ = model(inputs)

In [26]:
from models.mobilenetv2 import quat_mobilenet_v2
new = False
if new:
    quat_model = quat_mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1,activation_layer=nn.ReLU)
    print(quat_model)
    quat_model.classifier.append(nn.Dropout(0.2))
    quat_model.classifier.append(nn.Linear(1000, 10))
    quat_model.fuse_model()
else:
    pre_model = copy.deepcopy(fused_model)
    _,acc = Evaluating(pre_model,Test_loader,cpu_device)
    print(f"Before quantization acc : {acc:.2f} %")
    pre_model.eval()
    pre_model.qconfig = torch.quantization.get_default_qconfig("fbgemm")
    torch.quantization.prepare(pre_model,inplace=True)
    calibrate_model(pre_model, Test_loader)
    pre_model = torch.quantization.convert(pre_model,inplace=True)
    torch.jit.save(torch.jit.script(pre_model),"./models/Q_mobilenetv2_cifar10_jit.pt")
    pre_model = torch.jit.load("./models/Q_mobilenetv2_cifar10_jit.pt")
    _,int8_acc = Evaluating(pre_model,Test_loader,cpu_device)
    print(f"post int8_model acc :{int8_acc:.2f} %")
    
    quat_model = fused_model
# qconfig("fbgemm") 은 server 용 "qnnpack"은 mobile용 ["fbgemm", "x86", "qnnpack", "onednn"]

# QAT를 하기위해 quantization 모델 준비
quat_model.train()
quat_model.qconfig = torch.quantization.get_default_qconfig("fbgemm")
quat_model = torch.quantization.prepare_qat(quat_model)
print(type(quat_model))

# print('Inverted Residual Block: After preparation for QAT, note fake-quantization modules \n',quat_model.features[1].conv)

100%|██████████| 79/79 [00:04<00:00, 18.92it/s]
/home/seunmul/.conda/envs/torch/lib/python3.9/site-packages/torch/ao/quantization/observer.py:176: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Before quantization acc : 90.06 %
calibrating ...


100%|██████████| 79/79 [00:10<00:00,  7.83it/s]
/home/seunmul/.conda/envs/torch/lib/python3.9/site-packages/torch/ao/quantization/observer.py:1135: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(
100%|██████████| 79/79 [00:04<00:00, 16.17it/s]


post int8_model acc :85.65 %
<class 'models.mobilenetv2.QuantizableMobileNetV2'>


In [14]:
qat_need = True
if qat_need:
    optimizer = torch.optim.SGD(quat_model.parameters(), lr=1e-5, momentum=0.9, weight_decay=5e-4)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,60,90], gamma=0.5)

    first_loss, first_acc = Evaluating(model=quat_model,test_loader=Test_loader,device=cpu_device,criterion=nn.CrossEntropyLoss())
    print(f"Before Acc : {first_acc:.2f} acc")
    quat_model = Training(quat_model,train_loader=Train_loader,test_loader=Test_loader,
    device=gpu_device,optimizer=optimizer,scheduler=scheduler,epochs=30,model_name="QAT_mobilenetv2_cifar10")

100%|██████████| 79/79 [00:08<00:00,  9.06it/s]


Before Acc : 90.06 acc
Before Training
  Allocated: 0.0 GB
  Cached:    0.03 GB



100%|██████████| 79/79 [00:06<00:00, 11.54it/s]


--------0----------
Train 0.6539 Loss, 94.07 Acc
Validation 0.7404 Loss, 89.93 Acc


100%|██████████| 79/79 [00:06<00:00, 11.61it/s]


...

--------27----------
Train 0.6408 Loss, 94.65 Acc
Validation 0.7350 Loss, 90.31 Acc


100%|██████████| 79/79 [00:06<00:00, 11.60it/s]


--------28----------
Train 0.6400 Loss, 94.66 Acc
Validation 0.7348 Loss, 90.28 Acc


100%|██████████| 79/79 [00:06<00:00, 11.76it/s]

--------29----------
Train 0.6367 Loss, 94.84 Acc
Validation 0.7349 Loss, 90.33 Acc


In [15]:
# QAT가 적용된 floating point 모델을 quantized int model로 변환
# quat_model.load_state_dict(torch.load("./models/QAT_mobilenetv2_cifar10.pt"))
quat_model.to('cpu')
int8_model = torch.ao.quantization.convert(quat_model)
int8_model.eval()
_,int8_acc = Evaluating(int8_model,Test_loader,cpu_device)
print(f"int8_model acc : {int8_acc:.2f} %")
torch.jit.save(torch.jit.script(int8_model),"./models/QAT_mobilenetv2_cifar10_jit.pt")
int8_model = torch.jit.load("./models/QAT_mobilenetv2_cifar10_jit.pt",map_location=cpu_device)

100%|██████████| 79/79 [00:03<00:00, 24.29it/s]


int8_model acc : 88.47 %


In [16]:
_,int8_acc = Evaluating(int8_model,Test_loader,cpu_device)
print(f"jit int8_model acc : {int8_acc:.2f} %")

100%|██████████| 79/79 [00:04<00:00, 15.96it/s]

jit int8_model acc : 88.47 %


In [27]:
pre_model = torch.jit.load("./models/Q_mobilenetv2_cifar10_jit.pt")
_,int8_acc = Evaluating(pre_model,Test_loader,cpu_device)
print(f"post int8_model acc :{int8_acc:.2f} %")

100%|██████████| 79/79 [00:04<00:00, 16.03it/s]

post int8_model acc :85.65 %


In [17]:
from models import quat_mobilenet_v2
from torchsummary import summary
inverted_residual_setting = [
                # t, c, n, s
                [1, 16, 1, 1],
                # [6, 24, 2, 1],  # NOTE: change stride 2 -> 1 for CIFAR10
                [4, 32, 3, 2],
                [4, 64, 4, 2],
                # [6, 96, 3, 1],
                [4, 128, 3, 2],
                # [6, 320, 1, 1],
            ]
tiny_mobilenet = quat_mobilenet_v2(num_classes=10, inverted_residual_setting=inverted_residual_setting)
summary(tiny_mobilenet,(3,32,32),device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         QuantStub-1            [-1, 3, 32, 32]               0
            Conv2d-2           [-1, 32, 16, 16]             864
       BatchNorm2d-3           [-1, 32, 16, 16]              64
              ReLU-4           [-1, 32, 16, 16]               0
            Conv2d-5           [-1, 32, 16, 16]             288
       BatchNorm2d-6           [-1, 32, 16, 16]              64
              ReLU-7           [-1, 32, 16, 16]               0
            Conv2d-8           [-1, 16, 16, 16]             512
       BatchNorm2d-9           [-1, 16, 16, 16]              32
QuantizableInvertedResidual-10           [-1, 16, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]           1,024
      BatchNorm2d-12           [-1, 64, 16, 16]             128
             ReLU-13           [-1, 64, 16, 16]               0
           Conv2d-14         

In [18]:
optimizer = torch.optim.SGD(tiny_mobilenet.parameters(), lr=5e-3, momentum=0.9, weight_decay=5e-4)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,30,70], gamma=0.5)

tiny_mobilenet = Training(tiny_mobilenet,Train_loader,Test_loader,gpu_device,optimizer,scheduler)

Before Training
  Allocated: 0.01 GB
  Cached:    0.33 GB



100%|██████████| 79/79 [00:01<00:00, 41.24it/s]


--------1----------
Train 1.9500 Loss, 29.97 Acc
Validation 1.7236 Loss, 42.17 Acc


100%|██████████| 79/79 [00:01<00:00, 42.07it/s]


--------2----------
Train 1.7205 Loss, 42.56 Acc
Validation 1.6044 Loss, 48.58 Acc


...

--------98----------
Train 0.8563 Loss, 84.54 Acc
Validation 0.9347 Loss, 81.27 Acc


100%|██████████| 79/79 [00:01<00:00, 41.19it/s]


--------99----------
Train 0.8529 Loss, 84.81 Acc
Validation 0.9365 Loss, 81.22 Acc


100%|██████████| 79/79 [00:01<00:00, 41.20it/s]

--------100----------
Train 0.8526 Loss, 84.84 Acc
Validation 0.9326 Loss, 81.60 Acc


In [2]:
from models import quat_mobilenet_v2
NEW_MODEL = quat_mobilenet_v2(cifar10=True)
summary(NEW_MODEL,(3,32,32),device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         QuantStub-1            [-1, 3, 32, 32]               0
            Conv2d-2           [-1, 32, 16, 16]             864
       BatchNorm2d-3           [-1, 32, 16, 16]              64
              ReLU-4           [-1, 32, 16, 16]               0
            Conv2d-5           [-1, 32, 16, 16]             288
       BatchNorm2d-6           [-1, 32, 16, 16]              64
              ReLU-7           [-1, 32, 16, 16]               0
            Conv2d-8           [-1, 16, 16, 16]             512
       BatchNorm2d-9           [-1, 16, 16, 16]              32
QuantizableInvertedResidual-10           [-1, 16, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]           1,024
      BatchNorm2d-12           [-1, 64, 16, 16]             128
             ReLU-13           [-1, 64, 16, 16]               0
           Conv2d-14         